In [ ]:
# Personalizable Hierarchical Models

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import arviz as az

In [ ]:
# import and visualize data

MylanTDPDays100 = np.genfromtxt('Data/DailyMed_MylanTDP_n36/MylanTDP_n36_100mcg.csv', delimiter=',', skip_header=True)
MylanTDPDays50 = np.genfromtxt('Data/DailyMed_MylanTDP_n36/MylanTDP_n36_50mcg.csv', delimiter=',', skip_header=True)
MylanTDPDays25 = np.genfromtxt('Data/DailyMed_MylanTDP_n36/MylanTDP_n36_25mcg.csv', delimiter=',', skip_header=True)

H2003aN30Estradot100 = np.genfromtxt('Data/Hossain2003a_Estradot_n30/H2003a_Estradot_n30_100mcg.csv', delimiter=',', skip_header=True)
H2003aN30Estradot50 = np.genfromtxt('Data/Hossain2003a_Estradot_n30/H2003a_Estradot_n30_50mcg.csv', delimiter=',', skip_header=True)
H2003aN30Estradot37_5 = np.genfromtxt('Data/Hossain2003a_Estradot_n30/H2003a_Estradot_n30_37.5mcg.csv', delimiter=',', skip_header=True)
H2003aN30Estradot25 = np.genfromtxt('Data/Hossain2003a_Estradot_n30/H2003a_Estradot_n30_25mcg.csv', delimiter=',', skip_header=True)

H2003bN11Estradot100 = np.genfromtxt('Data/Hossain2003b_Estradot_n11/H2003b_Estradot_n11_100mcg.csv', delimiter=',', skip_header=True)
H2003bN11Estradot50 = np.genfromtxt('Data/Hossain2003b_Estradot_n11/H2003b_Estradot_n11_50mcg.csv', delimiter=',', skip_header=True)

# exclude n = 30 study from H2003b due to the initial measurement being higher than the second measurement
#H2003bN30Estradot50 = np.genfromtxt('Data/Hossain2003b_Estradot_n30/H2003b_Estradot_n30_50mcg.csv', delimiter=',', skip_header=True)

# convert Mylan data from days to hrs, Estradot data is already in hrs
# https://stackoverflow.com/questions/10394659/how-to-add-a-calculated-computed-column-in-numpy

MylanTDP100 = np.vstack((MylanTDPDays100[:,0]*24, MylanTDPDays100[:,1])).T
MylanTDP50 = np.vstack((MylanTDPDays50[:,0]*24, MylanTDPDays50[:,1])).T
MylanTDP25 = np.vstack((MylanTDPDays25[:,0]*24, MylanTDPDays25[:,1])).T

In [ ]:
def E2DoseTerm(t, d0, k1, k2, k3, es0=0.0, e20=0.0):
    innerFunc1 = np.exp(-k1 * t) / ((k1 - k2) * (k1 - k3))
    innerFunc2 = np.exp(-k2 * t) / ((k1 - k2) * (k2 - k3))
    innerFunc3 = np.exp(-k3 * t) / ((k1 - k3) * (k2 - k3))
    summationDose = d0 * k1 * k2 * (innerFunc1 - innerFunc2 + innerFunc3)
    if es0 > 0.0:
      summationDose += es0 * k2 / (k2 - k3) * (np.exp(-k3 * t) - np.exp(-k2 * t))
    if e20 > 0.0:
      summationDose += e20 * np.exp(-k3 * t)
    return summationDose

def Es(t, d0, k1, k2):
    return d0 * k1 / (k1 - k2) * (np.exp(-k2 * t) - np.exp(-k1 * t))

def E2Patch(t, d0, k1, k2, k3, W):    # W \equiv t_{\rm rem}
    if t < 0:
        return 0.0
    elif 0 <= t < W:
        return E2DoseTerm(t, d0, k1, k2, k3, es0=0.0, e20=0.0)
    elif t >= W:
        Es_at_W = Es(W, d0, k1, k2)
        E2_at_W = E2DoseTerm(W, d0, k1, k2, k3, es0=0.0, e20=0.0)
        return E2DoseTerm(t - W, 0.0, k1, k2, k3, es0=Es_at_W, e20=E2_at_W)

In [ ]:
# alternative to E2Patch, scalar version
def E2PatchLoop(t, d0, k1, k2, k3, W):
    E2Out = np.zeros(len(t))
    for i in range(len(t)):
        if t[i] < 0:
            E2Out[i] = 0.0
        elif 0 <= t[i] < W:
            E2Out[i] = E2DoseTerm(t[i], d0, k1, k2, k3, es0=0.0, e20=0.0)
        elif t[i] >= W:
            Es_at_W = Es(W, d0, k1, k2)
            E2_at_W = E2DoseTerm(W, d0, k1, k2, k3, es0=0.0, e20=0.0)
            E2Out[i] = E2DoseTerm(t[i] - W, 0.0, k1, k2, k3, es0=Es_at_W, e20=E2_at_W)
    return E2Out

In [ ]:
# alternative to E2Patch, wihtout if-elif statements, tensor version
def E2DoseTermFull(t, d0, k1, k2, k3, es0, e20):
    innerFunc1 = np.exp(-k1 * t) / ((k1 - k2) * (k1 - k3))
    innerFunc2 = np.exp(-k2 * t) / ((k1 - k2) * (k2 - k3))
    innerFunc3 = np.exp(-k3 * t) / ((k1 - k3) * (k2 - k3))
    summationDose = d0 * k1 * k2 * (innerFunc1 - innerFunc2 + innerFunc3)
    summationDose += es0 * k2 / (k2 - k3) * (np.exp(-k3 * t) - np.exp(-k2 * t))
    summationDose += e20 * np.exp(-k3 * t)
    return summationDose

# Visualise the Gamma and Inverse-Gamma Distributions

In [ ]:
# visualise gamma distribution
# https://www.pymc.io/projects/docs/en/latest/api/distributions/generated/pymc.Gamma.html
import scipy.stats as st

x = np.linspace(0, 4000, 1000)
mu = (1500+400)/2
sigma = 500

# the shape parameter, usually set to 1, 2, or 3, https://discord.com/channels/438306949285806082/1201973466845151362/1211192844182884422
# alpha = 0.5 exhibits 1/sqrt close to 0, alpha = 1 exhibits constant close to 0, ...
# ... alpha = 2 exhibits linear close to 0, alpha = 3 exhibits quadratic close to 0, https://discord.com/channels/438306949285806082/1201973466845151362/1211193057639407656
# https://media.discordapp.net/attachments/1201973466845151362/1211194297743970324/image.png?ex=65ed4f97&is=65dada97&hm=4d4967d75964b2dad982c1e8f7534f40b1f2f5545d20ff4d7594b1cd96aea104&=&format=webp&quality=lossless&width=1234&height=943
# the higher the alpha is, and the more strongly you suppress close to zero, the more you necessarily push the bulk of the distribution away from it
alpha = mu**2/sigma**2

beta = mu/sigma**2    # the rate parameter
pdf = st.gamma.pdf(x, a=alpha, scale=1.0/beta)

plt.plot(x, pdf, label=r'$\alpha$ = {}, $\beta$ = {}'.format(alpha, beta))
plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.legend(loc=1)
plt.show()

In [ ]:
# inverse gamma
# if X ~ Gamma(a, b), then 1/X ~ InverseGamma(a, 1/b)

x = np.linspace(0, 20, 1000)
mu = 1/(100/24)
sigma = 1

# the shape parameter, usually set to 1, 2, or 3, https://discord.com/channels/438306949285806082/1201973466845151362/1211192844182884422
# alpha = 0.5 exhibits 1/sqrt close to 0, alpha = 1 exhibits constant close to 0, ...
# ... alpha = 2 exhibits linear close to 0, alpha = 3 exhibits quadratic close to 0, https://discord.com/channels/438306949285806082/1201973466845151362/1211193057639407656
# https://media.discordapp.net/attachments/1201973466845151362/1211194297743970324/image.png?ex=65ed4f97&is=65dada97&hm=4d4967d75964b2dad982c1e8f7534f40b1f2f5545d20ff4d7594b1cd96aea104&=&format=webp&quality=lossless&width=1234&height=943
# the higher the alpha is, and the more strongly you suppress close to zero, the more you necessarily push the bulk of the distribution away from it
alpha = mu**2/sigma**2

beta = mu/sigma**2    # the rate parameter
pdf = st.invgamma.pdf(x, a=alpha, scale=beta)

plt.plot(x, pdf, label=r'$\alpha$ = {}, $\beta$ = {}'.format(alpha, beta))
plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.legend(loc=1)
plt.show()

# Hierarchical Model of Estradot and Mylan Data

In [ ]:
with pm.Model() as MylanMulti:

    # Mylan 100 mcg/day
    Mylan100TimeData = pm.ConstantData("Time_100", MylanTDP100[:,0], dims="obs_100")
    Mylan100Data = pm.ConstantData("E2Observed_100", MylanTDP100[:,1], dims="obs_100")
    sigmaMylan100 = pm.InverseGamma("sigma_100",alpha=3, beta=0.5)    # Standard deviation, infer from data

    # Mylan 50 mcg/day
    Mylan50TimeData = pm.ConstantData("Time_50", MylanTDP50[:,0], dims="obs_50")
    Mylan50Data = pm.ConstantData("E2Observed_50", MylanTDP50[:,1], dims="obs_50")
    sigmaMylan50 = pm.InverseGamma("sigma_50",alpha=3, beta=0.5)

    # Mylan 25 mcg/day
    Mylan25TimeData = pm.ConstantData("Time_25", MylanTDP25[:,0], dims="obs_25")
    Mylan25Data = pm.ConstantData("E2Observed_25", MylanTDP25[:,1], dims="obs_25")
    sigmaMylan25 = pm.InverseGamma("sigma_25",alpha=3, beta=0.5)

    # set patch removal time
    W = 3.5*24

    ### hyperpriors ###
    #d0GammaMu = pm.Gamma("d0_Gamma_mu", mu=(1500+400)/2, sigma=500)
    #d0GammaSigma = pm.Gamma("d0_Gamma_sigma", mu=500, sigma=500)
    # the above is when we don't know about d0 and it is one level too many in the hierarchy, https://discord.com/channels/438306949285806082/1201973466845151362/1211527645263564850

    d0 = pm.Gamma("d0", mu=(1500+400)/2, sigma=500)
    ### end hyperpriors ###

    ### priors on fitting parameters ###
    # you have a exp(-beta*x) in the distribution, so x's that become larger than 1/beta are suppressed exponentially ...
    # ... you want the prior on d0 to be pretty diffuse, so beta should be very small...
    # ... There's always a good ol' uniform prior with a very high cutoff for d0. https://discord.com/channels/438306949285806082/1201973466845151362/1210814524957921280
    d0_100 = d0    # pm.Gamma("d0_100", mu=d0GammaMu, sigma=d0GammaSigma)
    d0_50 = d0/2
    d0_25 = d0/4

    # the initial conditions Aeff0 and E20 depend on k1, https://discord.com/channels/438306949285806082/1201973466845151362/1209306363440930927
    # it's the beta of the gamma distribution that you should use to set the average, not the alpha. ...
    # ... Alpha controls the power-law behavior close to 0 and you'll get wonky stuff if you're setting it too small, https://discord.com/channels/438306949285806082/1201973466845151362/1210805371862646855
    k1 = pm.InverseGamma("k1", mu=1/(100/24), sigma=1)    # based on previous fits
    k2 = pm.InverseGamma("k2", mu=1, sigma=1)
    k3 = pm.InverseGamma("k3", mu=1/37, sigma=1)
    ### end priors ###

    # initial conditions after patch removal
    #Es_at_W = Es(W, d0, k1, k2)
    #E2_at_W = E2DoseTermFull(W, d0, k1, k2, k3, 0.0, 0.0)

    # likelihood
    # The likelihood function can be a lognormal but you have to be careful not to feed it 0 values unless you add a (random) positive...
    # ... baseline first, so like xi ~ LogNormal(log(E2(ti) + b), sigma)
    E2Hat_100 = pm.Deterministic("E2Hat_100", pm.math.switch(pm.math.lt(Mylan100TimeData, W),
                                                     E2DoseTermFull(Mylan100TimeData, d0_100, k1, k2, k3, 0.0, 0.0),
                                                     E2DoseTermFull(Mylan100TimeData - W, 0.0, k1, k2, k3,
                                                                    Es(W, d0_100, k1, k2),
                                                                    E2DoseTermFull(W, d0_100, k1, k2, k3, 0.0, 0.0))),
                                 dims="obs_100")
    Likelihood_100 = pm.Normal("Likelihood_100", mu = E2Hat_100, observed = Mylan100Data, sigma=sigmaMylan100, dims="obs_100")

    E2Hat_50 = pm.Deterministic("E2Hat_50", pm.math.switch(pm.math.lt(Mylan50TimeData, W),
                                                     E2DoseTermFull(Mylan50TimeData, d0_50, k1, k2, k3, 0.0, 0.0),
                                                     E2DoseTermFull(Mylan50TimeData - W, 0.0, k1, k2, k3,
                                                                    Es(W, d0_50, k1, k2),
                                                                    E2DoseTermFull(W, d0_50, k1, k2, k3, 0.0, 0.0))),
                                dims="obs_50")
    Likelihood_50 = pm.Normal("Likelihood_50", mu = E2Hat_50, observed = Mylan50Data, sigma=sigmaMylan50, dims="obs_50")

    E2Hat_25 = pm.Deterministic("E2Hat_25", pm.math.switch(pm.math.lt(Mylan25TimeData, W),
                                                     E2DoseTermFull(Mylan25TimeData, d0_25, k1, k2, k3, 0.0, 0.0),
                                                     E2DoseTermFull(Mylan25TimeData - W, 0.0, k1, k2, k3,
                                                                    Es(W, d0_25, k1, k2),
                                                                    E2DoseTermFull(W, d0_25, k1, k2, k3, 0.0, 0.0))),
                                dims="obs_25")
    Likelihood_25 = pm.Normal("Likelihood_25", mu = E2Hat_25, observed = Mylan25Data, sigma=sigmaMylan25, dims="obs_25")

# visualize the model
pm.model_to_graphviz(MylanMulti)

In [ ]:
# diagnostic plot
timeSS_ = np.arange(0,5*24,0.1)
E2PatchOut = np.zeros(len(timeSS_))
pMean = np.array([az.summary(sampleMylan100Removal3, var_names = ["d0"], kind="stats").values[0][0],
                  az.summary(sampleMylan100Removal3, var_names = ["k1"], kind="stats").values[0][0],
                  az.summary(sampleMylan100Removal3, var_names = ["k2"], kind="stats").values[0][0],
                  az.summary(sampleMylan100Removal3, var_names = ["k3"], kind="stats").values[0][0]])
for i in range(len(timeSS_)):
    E2PatchOut[i] = E2Patch(timeSS_[i], *pMean, 3.5*24)
plt.plot(timeSS_, E2PatchOut)